In [1]:
#Imports
import pandas as pd
from collections import defaultdict, OrderedDict

In [2]:
#Data Reading and Formatting
directory = r'(filepath to data)'
OD_readings = pd.read_excel('%s\input_data.xlsx'%directory, sheet_name = 'OD_readings')
MIC_metadata = pd.read_excel('%s\input_data.xlsx'%directory, sheet_name = 'MIC_metadata')
OD_readings = OD_readings.reset_index(drop=True).set_index('Drugs')
MIC_metadata = MIC_metadata.reset_index(drop=True).set_index('Drugs').T

In [3]:
#Grouping the Replicates
grouped_metadata = defaultdict(dict)
for antibiotic in MIC_metadata.columns[:11]:
    for concentration in set(MIC_metadata[antibiotic]):
        grouped_metadata[antibiotic][concentration] = MIC_metadata.loc[MIC_metadata[antibiotic] == concentration].index.tolist()

In [4]:
#Grouping the Readings
grouped_readings = defaultdict(dict)
for antibiotic in OD_readings.index:
    for concentration in grouped_metadata[antibiotic].keys():
        columns = grouped_metadata[antibiotic][concentration] 
        grouped_readings[antibiotic][concentration] = OD_readings.loc[antibiotic, columns].values
grouped_readings = {antibiotic: OrderedDict(sorted(d.items(), key = lambda a: a[0])) for antibiotic, d in grouped_readings.items()}

In [5]:
#Function to treat special cases
def treat_special(reading_array, cutoff = 0.15, minimum = 0.135, maximum = 0.165):
    if(sum(reading_array>cutoff)==2):
        if(reading_array[reading_array <= cutoff][0]>=minimum):
            if(i+1 < len(readings)):
                MIC = '%s'%str(list(readings.keys())[i+1])
            else:
                MIC = '>%s'%str(key)
        else:
            if(i+1 < len(readings)):
                MIC = '%s-%s'%(str(list(readings.keys())[i]), float(list(readings.keys())[i+1]))
            else:
                MIC = '%s'%str(list(readings.keys())[i])
    elif(sum(reading_array>cutoff)==1):
        if(reading_array[reading_array > cutoff][0]<maximum):
            MIC = '%s'%str(list(readings.keys())[i])
        else:
            if(i+1 < len(readings)):
                MIC = '%s-%s'%(str(list(readings.keys())[i]), float(list(readings.keys())[i+1]))
            else:
                MIC = '>%s'%str(list(readings.keys())[i])
    return(MIC)

In [6]:
#Finding the MICs
cutoff = 0.15
minimum = 0.135
maximum = 0.165
resulting_MICs = {}
for antibiotic, readings in grouped_readings.items():
    res = list(readings.keys())
    for i, key in enumerate(readings.keys()):
        reading_concentration = list(readings.keys())[i]
        reading_array = readings[key]
        #print(reading_concentration ,(reading_array>cutoff))
        if(reading_array>cutoff).all():
            res.remove(reading_concentration)
            #print('All true')
        elif not(reading_array<=cutoff).all():
            #print('special')
            resulting_MICs[antibiotic] = treat_special(reading_array)
            break
        #print(antibiotic,res)

        if len(res) == 0:
             resulting_MICs[antibiotic] = '>%s'%str(max(readings.keys()))

        else:      
            resulting_MICs[antibiotic] = '%s'%str(min(res))

In [7]:
#Output to CSV
human_readable = pd.DataFrame(list(resulting_MICs.items()), columns = ['Antibiotic', 'MIC'])
human_readable = human_readable.set_index('Antibiotic').loc[MIC_metadata.columns.tolist()].reset_index().drop_duplicates()
human_readable.to_csv('%s\MICs for (Insert Name).csv'%directory, index=None)